# PREDICCION DE FASE USANDO NPL

## 1. Preparacion

### 1.1 Importe De Librerias

In [5]:
import pandas as pd
import numpy as np
from numpy import loadtxt
from pylab import *
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, HTML
from scipy import stats
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from statistics import mean

import re
import nltk
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from sklearn import preprocessing
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.model_selection import RandomizedSearchCV
import pickle
import nltk
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix

[nltk_data] Downloading package stopwords to C:\Users\Diego
[nltk_data]     Ojeda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
def pre_pro_well(base,well):
    '''
    FUNCION QUE SELECCIONA UN POZO, TRANSFORMA LOS DATOS COMO INPUT DEL MODELO

    - base: Dataframe con los datos de todos los pozos.
    - well: cadena de caracteres con el nombre del pozo a evaluar    
    '''

    # Seleccion del pozo a preprocesar
    base = base[base["Nombre"]==well]

    # Seleccion de la columna de tesxto
    X = base["Operación"]

    # Libreria de stopwords en ingles
    nltk.download('wordnet')

    # Lista de textos vacia
    documents = []
    
    # Instanciacion del lematizador en ingles
    stemmer = WordNetLemmatizer()

    # Ciclo for para que recorra todos las filas con las actividaes (textos)
    for sen in range(0, len(X)):

        # Removemos los caracteres especiales
        document = re.sub(r'\W', ' ', str(X[sen]))
        
        # Removemos todos las cadenas de caracteres de un solo caracter
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        
        # Removemos caracteres sencillso al inicio
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
        
        # Substituimos dobles espacios por espacios sencillos
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removemos caracteres numericos
        document = re.sub(r'\[[0-9]*\]',' ',document)
        
        # Convertimos todo a minusculas
        document = document.lower()
        
        # Realizamos Lematizacion
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)
        
        documents.append(document)
    
    return (documents,base)

def identificacion_OCM(well):
    '''
    Función que usa un modelo de NPL para identificar las actividades y encontrar el inicio de OCM
    ----------------------------------------------------------------------------------------------

    - well = Cadena de caracteres con el nombre del pozo a probar

    Retorna
    - 
    '''
    #-----------------------------------------------------------------------------
    # LECTURA Y PREPARACION DE DATOS
    #-----------------------------------------------------------------------------

    # Lectura de Datos
    path='Data.xlsx'
    base = pd.read_excel(path)
    data = base

    # Eliminacion de columnas innecesarias y cambio de nombre de columnas
    data = data.rename({'Operación':"Operacion"}, axis=1)

    #-----------------------------------------------------------------------------
    # CARGA DE MODELOS
    #-----------------------------------------------------------------------------

    # Cargue del modelo entrenado
    model_rf = pickle.load(open("rf.sav", 'rb'))

    # Cargue del vectorizador entrenado
    vectorizer = pickle.load(open("vectorizer.sav", 'rb'))

    # Cargue del codificador de etiquetas
    le = pickle.load(open("label_encoder.sav", 'rb'))

    #-----------------------------------------------------------------------------
    # PREPROCESAMIENTO DE LA INFORMACION PARA LA PREDICCION DEL POZO SOLICITADO
    #-----------------------------------------------------------------------------

    # Preparamos los datos para la prediccion sobre un pozo especifico
    X_val,base_val = pre_pro_well(base,well)

    # Usamos el vectorizador para transformar los datos para la prediccion
    X_val = vectorizer.transform(X_val)

    #-----------------------------------------------------------------------------
    # PREDICCION SOBRE DATOS PREPROCESADOS
    #-----------------------------------------------------------------------------

    # Prediccion
    y_val = model_rf.predict(X_val)

    #-----------------------------------------------------------------------------
    # VISUALIZACION DE LA PREDICCION
    #-----------------------------------------------------------------------------

    # Union con datos del pozo
    base_val["Target"] = le.inverse_transform(y_val)

    # Organizacion del dataframe
    base_val = base_val[['Siglas', 'Target', 'Nombre', 'Desde', 'Hasta',"MD From (ft)","MD to (ft)","Codigo","Subcodigo","Operación"]]
    
    #-----------------------------------------------------------------------------
    # BUSQUEDA DE INICIO DE OCM
    #-----------------------------------------------------------------------------

    # Organizacion de la base en orden descendente
    base_val = base_val.sort_values(by=['Desde'],ascending=False)
    base_val = base_val.reset_index(drop=True)

    # Algoritmo para deteccion de inicio de OCM
    fila = 1
    while (base_val.Target[fila] == base_val.Target[fila-1]):
        fila=fila+1
    else:
        ID_OCM=fila-1

    OCM_Start = base_val.Desde[ID_OCM]

    base_val = base_val.sort_values(by=['Desde'],ascending=True)
    base_val = base_val.reset_index(drop=True)
    
    print("El Inicio De OCM En El Pozo " + str(well) + " Se Da En La Fecha: " + str(OCM_Start))
    return(base_val[-25:])

In [22]:
identificacion_OCM("MOR BL 04")

El Inicio De OCM En El Pozo MOR BL 04 Se Da En La Fecha: 2011-05-25 02:30:00


[nltk_data] Downloading package wordnet to C:\Users\Diego
[nltk_data]     Ojeda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\DIEGOO~1\AppData\Local\Temp/ipykernel_16032/4111610095.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_val["Target"] = le.inverse_transform(y_val)


,Siglas,Target,Nombre,Desde,Hasta,MD From (ft),MD to (ft),Codigo,Subcodigo,Operación
87,ODR,ODR,MOR BL 04,2011-05-23 03:00:00,2011-05-23 08:00:00,2773.0,2773.0,1200,1206.0,"CONTD. RIH W/ 3½"" DP + GRAVEL PACKER TOOL + 5½..."
88,ODR,ODR,MOR BL 04,2011-05-23 08:00:00,2011-05-23 09:00:00,2773.0,2773.0,1500,1503.0,CIRCULATED @ 2770 FT CONDITIONING MUD TO PERFO...
89,ODR,ODR,MOR BL 04,2011-05-23 09:00:00,2011-05-23 14:00:00,2773.0,2773.0,1500,1503.0,"GRAVEL PACKING 5½"" LINER W/ 10-16 MESH GRAVEL ..."
90,ODR,ODR,MOR BL 04,2011-05-23 14:00:00,2011-05-23 14:30:00,2773.0,2773.0,1500,1503.0,CIRCULATED IN DIRECT WAY W/37 SPM-120 GPM TO F...
91,ODR,ODR,MOR BL 04,2011-05-23 14:30:00,2011-05-23 18:30:00,2773.0,2773.0,1500,1503.0,"GRAVEL PACKING 5½"" LINER W/ 10-16 MESH GRAVEL ..."
92,ODR,ODR,MOR BL 04,2011-05-23 18:30:00,2011-05-23 19:00:00,2773.0,2773.0,1500,1503.0,CIRCULATED IN DIRECT WAY W/37 SPM-120 GPM TO F...
93,ODR,ODR,MOR BL 04,2011-05-23 19:00:00,2011-05-23 23:30:00,2773.0,2773.0,1500,1503.0,"GRAVEL PACKING 5½"" LINER W/ 10-16 MESH GRAVEL ..."
94,ODR,ODR,MOR BL 04,2011-05-23 23:30:00,2011-05-24 00:00:00,2773.0,2773.0,1500,1503.0,CIRCULATED IN DIRECT WAY W/37 SPM-120 GPM TO F...
95,ODR,ODR,MOR BL 04,2011-05-24 00:00:00,2011-05-24 03:30:00,2773.0,2773.0,1500,1503.0,"GRAVEL PACKING 5½"" LINER W/ 10-16 MESH GRAVEL ..."
96,ODR,ODR,MOR BL 04,2011-05-24 03:30:00,2011-05-24 04:00:00,2773.0,2773.0,1500,1503.0,CIRCULATED IN DIRECT WAY W/37 SPM-120 GPM TO F...
